# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import hvplot.xarray  # noqa
import xarray as xr
import hvplot.pandas
import holoviews as hv
from bokeh.io import show
# Import API key
from api_keys import KEY


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,298.28,77,94,4.48,YE,1732229294
1,1,rio grande,-32.0350,-52.0986,293.32,72,100,8.94,BR,1732229237
2,2,rabo de peixe,37.8000,-25.5833,291.88,99,20,4.12,PT,1732229294
3,3,adamstown,-25.0660,-130.1015,296.22,88,40,7.10,PN,1732229294
4,4,as sanamayn,33.0723,36.1838,286.26,68,27,0.39,SY,1732229294


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = 'OSM',
    color = "City",
    size = 'Humidity'
)
# Display the map
# YOUR CODE HERE
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
df_filter = city_data_df[(city_data_df['Humidity'] > 70) & (city_data_df['Humidity'] < 90)]
# Drop any rows with null values
# YOUR CODE HERE
df_clean = df_filter.dropna()
# Display sample data
# YOUR CODE HERE
df_clean.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,298.28,77,94,4.48,YE,1732229294
1,1,rio grande,-32.0350,-52.0986,293.32,72,100,8.94,BR,1732229237
3,3,adamstown,-25.0660,-130.1015,296.22,88,40,7.10,PN,1732229294
6,6,port-aux-francais,-49.3500,70.2167,276.87,77,41,17.13,TF,1732229294
12,12,howrah,22.5892,88.3103,294.15,78,20,3.60,IN,1732229295


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = df_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel_name'] = ''
# Display sample data
# YOUR CODE HERE
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel_name
0,hadibu,YE,12.6500,54.0333,77,
1,rio grande,BR,-32.0350,-52.0986,72,
3,adamstown,PN,-25.0660,-130.1015,88,
6,port-aux-francais,TF,-49.3500,70.2167,77,
12,howrah,IN,22.5892,88.3103,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
PARAMS = {
    "categories": "accommodation.hotel",
    "limit":20,
    "apiKey": KEY
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    lng = row['Lng']
    # Add the current city's latitude and longitude to the params dictionary
    PARAMS["filter"] = f"circle:{lat},{lng},{radius}"
    PARAMS["bias"] = f"proximity:{lat},{lng}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=PARAMS)
    print(response)

        # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel_name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel_name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel_name']}")

# Display sample data
hotel_df.head()


Starting hotel search
<Response [200]>
hadibu - nearest hotel: Gut Nustrow
<Response [200]>
rio grande - nearest hotel: No hotel found
<Response [200]>
adamstown - nearest hotel: No hotel found
<Response [200]>
port-aux-francais - nearest hotel: No hotel found
<Response [200]>
howrah - nearest hotel: No hotel found
<Response [200]>
droichead nua - nearest hotel: No hotel found
<Response [200]>
anadyr - nearest hotel: No hotel found
<Response [200]>
ouani - nearest hotel: No hotel found
<Response [200]>
atafu village - nearest hotel: No hotel found
<Response [200]>
cururupu - nearest hotel: No hotel found
<Response [200]>
cadale - nearest hotel: Est Ouest Hotel
<Response [200]>
rolante - nearest hotel: No hotel found
<Response [200]>
aljezur - nearest hotel: No hotel found
<Response [200]>
la passe - nearest hotel: No hotel found
<Response [200]>
blackmans bay - nearest hotel: No hotel found
<Response [200]>
sihanoukville - nearest hotel: No hotel found
<Response [200]>
iwanai - nearest

,City,Country,Lat,Lng,Humidity,Hotel_name
0,hadibu,YE,12.6500,54.0333,77,Gut Nustrow
1,rio grande,BR,-32.0350,-52.0986,72,No hotel found
3,adamstown,PN,-25.0660,-130.1015,88,No hotel found
6,port-aux-francais,TF,-49.3500,70.2167,77,No hotel found
12,howrah,IN,22.5892,88.3103,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:


# Configure the map plot
# YOUR CODE HERE
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 1000,
    scale = 20,
    color = "City",
    hover_cols = ["Hotel_name", "Country"]
)

# YOUR CODE HERE
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel_name,Country)